In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sb
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,Dropout
from keras.layers import Flatten
from keras.optimizers import Adam

In [7]:
train = pd.read_csv('sign_mnist_train.csv')
test = pd.read_csv('sign_mnist_test.csv')

In [8]:
x_test = test.iloc[:, 1:].values
y_test = test.iloc[:, 0].values

In [9]:
x_train = train.iloc[:, 1:].values
y_train= train.iloc[:, 0].values

In [10]:
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

In [11]:
x_test=x_test/255.0
x_train=x_train/255.0

In [12]:
y_test = to_categorical(y_test, 26)
y_train = to_categorical(y_train, 26)

In [13]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first"))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(26, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [22]:
model_log = model.fit(x_train, y_train,batch_size=128,epochs=5,verbose=1)

Epoch 1/5
27455/27455 [==============================] - 97s 4ms/step - loss: 0.0426 - accuracy: 0.9894
Epoch 2/5
27455/27455 [==============================] - 93s 3ms/step - loss: 0.0331 - accuracy: 0.9914
Epoch 3/5
27455/27455 [==============================] - 94s 3ms/step - loss: 0.0363 - accuracy: 0.9907
Epoch 4/5
27455/27455 [==============================] - 94s 3ms/step - loss: 0.0218 - accuracy: 0.9944
Epoch 5/5
27455/27455 [==============================] - 93s 3ms/step - loss: 0.0259 - accuracy: 0.9938


In [33]:
y_pred=model.predict(x_test)

In [34]:
for j in range(7172):
    max_id=0
    for i in range (26):
        if y_pred[j][i]>y_pred[j][max_id]:
            max_id=i
    y_pred[j][max_id]=1
    for i in range (26):
        if i!=max_id:
            y_pred[j][i]=0

In [31]:
y_test[9]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [35]:
y_pred[9]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [36]:
from sklearn.metrics import mean_squared_error, r2_score
print(r2_score(y_test, y_pred))

0.945289874364263


In [37]:
letter_json = model.to_json()
with open("letter.json", "w") as json_file:
    json_file.write(letter_json)
model.save_weights("letter.h5")

In [38]:
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

In [39]:
img=cv2.imread('1.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
resize_img=cv2.resize(gray,(28,28))
resize_img= np.expand_dims(resize_img, axis =0)
resize_img = np.expand_dims(resize_img, axis=3)
pre=model.predict(resize_img)
max_id=0
for i in range (26):
    if pre[0][i]>pre[0][max_id]:
        max_id=i
emotions = ('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z')
print(emotions[max_id])

R


In [40]:
cap=cv2.VideoCapture(0)
while True:
    cp,img=cap.read()
    if not cp:
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resize_img=cv2.resize(gray,(28,28))
    resize_img= np.expand_dims(resize_img, axis =0)
    resize_img = np.expand_dims(resize_img, axis=3)
    pre=model.predict(resize_img)
    max_id=0
    for i in range (26):
        if pre[0][i]>pre[0][max_id]:
            max_id=i
    emotions = ('A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z')
    predicted_emotion = emotions[max_id]
    cv2.putText(img, predicted_emotion, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    resized_img = cv2.resize(img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)
    if cv2.waitKey(10) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows

<function destroyAllWindows>